In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np 
import pandas as pd 

def answer_similarity(answer, gt:list): 
    avg = [] 
    # print(gt, answer)

    for gta in gt : 
        gta = gta['answer'] 
        embeddings = encoder.encode([answer, gta]) 
        # print(embeddings.shape) # [3, 384]
        similarities = encoder.similarity(embeddings, embeddings)
        avg.append(similarities[1,0]) 
        # print(similarities) 

    return np.mean(avg) 
    
encoder = SentenceTransformer("all-MiniLM-L6-v2").to('cuda')

/home/work/.anaconda/envs/privacyovod_new_copy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/work/.anaconda/envs/privacyovod_new_copy/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [61]:
import os 
import sys
import json  
sys.path.append('/home/work/yuna/hpa/results')
sys.path.append('/home/work/yuna/VLMEval') 
from dataset.dataloader import PostProcessor # contractions, numbers, punctuations

from analysis.pilot.analysis import vqa_score, str_to_list 
from analysis.utils.question_type_mapper import question_type 
from utils import get_answer, get_question_id 

postprocessor = PostProcessor()  

def get_question_id(df, allqs=allqs): 
    allqs['question_id'] = allqs['question_id'].astype('int') 
    allqs = allqs.drop_duplicates(subset=['question'], keep='first') 

    return pd.merge(df, allqs, on=['question'], how='left') 

In [68]:

annotations_path = "/home/work/yuna/VLMEval/data/v2_mscoco_val2014_annotations.json"
with open(os.path.join(annotations_path), 'r') as f: 
    annotations = json.load(f)['annotations'] 

def get_answer(question_id):
    adf = pd.DataFrame(annotations)
    df['question_id'] = df['question_id'].astype(int) 
    target_row = df[df['question_id'] == question_id]
    return target_row.iloc[0]['answers']


Exception ignored in: <function tqdm.__del__ at 0x71e3895f1d80>
Traceback (most recent call last):
  File "/home/work/.anaconda/envs/privacyovod_new_copy/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/home/work/.anaconda/envs/privacyovod_new_copy/lib/python3.10/site-packages/tqdm/std.py", line 1267, in close
    if self.disable:
AttributeError: 'tqdm' object has no attribute 'disable'


In [37]:

with open(os.path.join("/home/work/yuna/VLMEval/data/v2_OpenEnded_mscoco_val2014_questions.json"), 'r') as f: 
    annotations = json.load(f)
allqs = annotations['questions'] 
allqs = pd.DataFrame(allqs)

In [ ]:
from models import gpt, gemini, gemini2_5

with open('/home/work/yuna/hpa/results/closed_source/gemini.jsonl', 'w', encoding='utf-8') as f: 
    for item in gemini: 
        f.write(json.dumps(item, ensure_ascii=False) + '\n')  

with open('/home/work/yuna/hpa/results/closed_source/gemini2_5.jsonl', 'w', encoding='utf-8') as f: 
    for item in gemini2_5: 
        f.write(json.dumps(item, ensure_ascii=False) + '\n')  

answers = []
for k,v in gpt.items(): 
    answers.append({
        'question': k, 
        'raw_output': v, 
        'model': "GPT5"
    })

with open('/home/work/yuna/hpa/results/closed_source/gpt5v.jsonl', 'w', encoding='utf-8') as f: 
    for item in answers: 
        f.write(json.dumps(item, ensure_ascii=False) + '\n')  
print('/home/work/yuna/hpa/results/closed_source/gpt5v.jsonl')

In [69]:
annotations_path = "/home/work/yuna/VLMEval/data/v2_mscoco_val2014_annotations.json"
with open(os.path.join(annotations_path), 'r') as f: 
    annotations = json.load(f)['annotations'] 
question_ids = pd.DataFrame(annotations)

def get_answer(question_id, df =question_ids):
    
    df['question_id'] = df['question_id'].astype(int) 
    target_row = df[df['question_id'] == question_id]
    return target_row.iloc[0]['answers']

In [ ]:
from utils import missing 
allqs = pd.concat([allqs, pd.DataFrame(missing)]) 
allqs 

In [72]:
from glob import glob
import pandas as pd
import os

# Ensure output directory exists
output_dir = '/home/work/yuna/hpa/results/closed_source-processed'
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

dfs = []
for f in glob('/home/work/yuna/hpa/results/closed_source/*.jsonl'):
    # Read JSONL file into DataFrame
    df = pd.read_json(f, lines=True)
    filename = f.split('/')[-1][:-6]  # Remove '.jsonl'
    output_file = os.path.join(output_dir, f"{filename}.jsonl")

    # Process DataFrame
    df = get_question_id(df)  # Ensure this function returns a DataFrame

    if 'answer' in df.columns:
        df.rename(columns={'answer': 'raw_output'}, inplace=True)

    if 'model' not in df.columns:
        df['model'] = filename
    # print(df)

    # Open output file
    with open(output_file, "w", encoding="utf-8") as fout:
        for i, row in df.iterrows():
            qid = int(row.get("question_id", 0))  # Default to 0 if missing
            ans = get_answer(qid)  # Ensure this function is defined
            processed_output = postprocessor.postprocess_answer(row.get("raw_output", ""))
            acc = vqa_score(processed_output, ans)  # Ensure this function is defined
            score= 0 
            try:
                score = float(answer_similarity(processed_output, ans))  # Ensure this function is defined
            except Exception as e:
                print(f"Error processing row {row}")
                continue
            item = {
                "qid": qid,
                "question": row.get("question", "").strip(),
                "question_type": question_type.get(qid, ""),  # Ensure question_type is defined
                "model": row.get("model", "").strip(),
                "raw_output": row.get("raw_output", "").strip(),
                "processed_output": processed_output,
                "acc": acc,
                "score": float(score)
            }
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"Saved to: {output_file}")
    dfs.append(df)

Saved to: /home/work/yuna/hpa/results/closed_source-processed/gpt5v.jsonl
Saved to: /home/work/yuna/hpa/results/closed_source-processed/gemini.jsonl
Saved to: /home/work/yuna/hpa/results/closed_source-processed/gemini2_5.jsonl


In [13]:
import json 
unique_questions_list = df.question.unique().tolist()
with open('./questions.py', 'w', encoding='utf-8') as f:
    json.dump(unique_questions_list, f, ensure_ascii=False) # , indent=4
    f.write('\n')  